In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib
import os
from surgeNN.surgeNN.evaluation import rmse, compute_precision, compute_recall, compute_f1, add_error_metrics_to_prediction_ds
from surgeNN.surgeNN.preprocessing import deseasonalize_da
from scipy.stats import rankdata

2025-05-08 07:50:50.515975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-08 07:50:50.592235: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#configure the script
tgs        = ['stavanger-svg-nor-nhs.csv','wick-wic-gbr-bodc.csv','esbjerg-esb-dnk-dmi.csv','immingham-imm-gbr-bodc.csv','den_helder-denhdr-nld-rws.csv', 'fishguard-fis-gbr-bodc.csv',  'brest-822a-fra-uhslc.csv', 'vigo-vigo-esp-ieo.csv',  'alicante_i_outer_harbour-alio-esp-da_mm.csv']
tgnames = ['Stavanger (NOR)','Wick (UK)', 'Esbjerg (DK)','Immingham (UK)','Den Helder (NL)','Fishguard (UK)','Brest (FR)','Vigo (PT)', 'Alicante (SP)']

qnts = np.array([.95,.98,.99,.995]) #quantiles, don't touch

Remove 'i' dimension from obs if n_steps is constant (this is the only factor that could change obs for different i)

**be careful as this overwrites results!**

In [3]:
path = '/home/jovyan/test_surge_models/results/nns/performance/lstm'
files = os.listdir(path)
for f in files:
    if f.startswith('.') == False:
        print('current_file:' +f)
        ds = xr.open_dataset(os.path.join(path,f))

        if len(np.unique(ds.hyperparameters.sel(p='n_steps').isel(split=0,drop=True)))==1: #all nsteps equal
            ds['o'] = ds['o'].isel(i=0,drop=True)

            ds.to_netcdf(os.path.join(path,f.replace('.nc','_v2.nc')),mode='w') #store new dataset
            del ds
            os.remove(os.path.join(path,f)) #delete old file
            os.rename(os.path.join(path,f.replace('.nc','_v2.nc')),os.path.join(path,f))#rename new file

current_file:lstm_3h_den_helder-denhdr-nld-rws_mse_hp1_it3.nc
current_file:lstm_3h_immingham-imm-gbr-bodc_mse_hp1_it2.nc
current_file:lstm_3h_vigo-vigo-esp-ieo_mse_hp1_it2.nc
current_file:lstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_it2.nc
current_file:lstm_3h_wick-wic-gbr-bodc_mse_hp1_it1.nc
current_file:lstm_3h_immingham-imm-gbr-bodc_mse_hp1_it3.nc
current_file:lstm_3h_vigo-vigo-esp-ieo_mse_hp1_it3.nc
current_file:lstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_it3.nc
current_file:lstm_3h_wick-wic-gbr-bodc_mse_hp1_it0.nc
current_file:lstm_3h_wick-wic-gbr-bodc_mse_hp1_it4.nc
current_file:lstm_3h_den_helder-denhdr-nld-rws_mse_hp1_it2.nc
current_file:lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it2.nc
current_file:lstm_3h_fishguard-fis-gbr-bodc_mse_hp1_ndeg1.5_it4.nc
current_file:lstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_ndeg1.5_it3.nc
current_file:lstm_3h_fishguard-fis-gbr-bodc_mse_hp1_it2.nc
current_file:lstm_3h_brest-822a-fra-uhslc_mse_hp1_it3.nc
current_file:lstm_3h_fishguard-fis-gbr-bodc_mse_hp1_n